In [2]:
%env THEANO_FLAGS=device=cpu
import os
from os import walk
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy.linalg as linalg

from PIL import Image

import theano
import theano.tensor as T

env: THEANO_FLAGS=device=cpu


In [34]:
'''
Implement the functions that were not implemented and complete the
parts of main according to the instructions in comments.
'''

def reconstructed_image(D,c,num_coeffs,X_mean,im_num):
    '''
    This function reconstructs an image given the number of
    coefficients for each image specified by num_coeffs
    '''
    
    '''
        Parameters
    ---------------
    c: np.ndarray
        a n x m matrix  representing the coefficients of all the image blocks.
        n represents the maximum dimension of the PCA space.
        m is (number of images x n_blocks**2)

    D: np.ndarray
        an N x n matrix representing the basis vectors of the PCA space
        N is the dimension of the original space (number of pixels in a block)

    im_num: Integer
        index of the image to visualize

    X_mean: np.ndarray
        a matrix representing the mean block.

    num_coeffs: Integer
        an integer that specifies the number of top components to be
        considered while reconstructing
    '''
    
    c_im = c[:num_coeffs,im_num]
    D_im = D[:,:num_coeffs]
    
    #TODO: Enter code below for reconstructing the image
    #......................
    #......................
    #X_recon_img = ........
    X_recon_img = np.dot(c_im.T,D_im.T).reshape(256,256) + X_mean

    return X_recon_img

In [36]:
def plot_reconstructions(D,c,num_coeff_array,X_mean,im_num):
    '''
    Plots 9 reconstructions of a particular image using D as the basis matrix and coeffiecient
    vectors from c

    Parameters
    ------------------------
        num_coeff_array: Iterable
            an iterable with 9 elements representing the number_of coefficients
            to use for reconstruction for each of the 9 plots
        
        c: np.ndarray
            a l x m matrix  representing the coefficients of all blocks in a particular image
            l represents the dimension of the PCA space used for reconstruction
            m represents the number of blocks in an image

        D: np.ndarray
            an N x l matrix representing l basis vectors of the PCA space
            N is the dimension of the original space (number of pixels in a block)

        X_mean: basis vectors represent the divergence from the mean so this
            matrix should be added to all reconstructed blocks

        im_num: Integer
            index of the image to visualize
    '''
    f, axarr = plt.subplots(3,3)
    for i in range(3):
        for j in range(3):
            plt.axes(axarr[i,j])
            plt.imshow(reconstructed_image(D,c,num_coeff_array[i*3+j],X_mean,im_num))
            
    f.savefig('outputb/hw1b_{0}.png'.format(im_num))
    plt.close(f)

In [5]:
def plot_top_16(D, sz, imname):
    '''
    Plots the top 16 components from the basis matrix D.
    Each basis vector represents an image block of shape (sz, sz)

    Parameters
    -------------
    D: np.ndarray
        N x n matrix representing the basis vectors of the PCA space
        N is the dimension of the original space (number of pixels in a block)
        n represents the maximum dimension of the PCA space (assumed to be atleast 16)

    sz: Integer
        The height and width of each block

    imname: string
        name of file where image will be saved.
    '''
    #TODO: Obtain top 16 components of D and plot them
    
    f, axarr = plt.subplots(4,4)
    for i in range(16):
        plt.axes(axarr[int(np.floor(i/4)),np.mod(i,4)])
        plt.imshow(D.T[i].reshape(sz,sz))
            
    f.savefig(imname)
    plt.close(f)

In [31]:
for path, dirs, files in os.walk('Fei_256'):
    l = len(files)
Ims = np.zeros((l,256*256))
for i,name in zip(range(l),files):
    Ims[i] = np.array(Image.open('Fei_256/'+ name), dtype= float).flatten()
Ims = Ims.astype(np.float32)
X_mn = np.mean(Ims, 0)
X = Ims - np.repeat(X_mn.reshape(1, -1), Ims.shape[0], 0)


In [7]:
len(X[1])

65536

In [51]:
def main():
    '''
    Read here all images(grayscale) from Fei_256 folder and collapse 
    each image to get an numpy array Ims with size (no_images, height*width).
    Make sure the images are read after sorting the filenames
    '''
    for path, dirs, files in os.walk('Fei_256'):
        l = len(files)
    Ims = np.zeros((l,256*256))
    for i,name in zip(range(l),files):
        Ims[i] = np.array(Image.open('Fei_256/'+ name), dtype= float).flatten()
    #TODO: Write a code snippet that performs as indicated in the above comment
    
    Ims = Ims.astype(np.float32)
    X_mn = np.mean(Ims, 0)
    X = Ims - np.repeat(X_mn.reshape(1, -1), Ims.shape[0], 0)

    '''
    Use theano to perform gradient descent to get top 16 PCA components of X
    Put them into a matrix D with decreasing order of eigenvalues

    If you are not using the provided AMI and get an error "Cannot construct a ufunc with more than 32 operands" :
    You need to perform a patch to theano from this pull(https://github.com/Theano/Theano/pull/3532)
    Alternatively you can downgrade numpy to 1.9.3, scipy to 0.15.1, matplotlib to 1.4.2
    '''
    

    d = theano.shared(np.random.randn(len(X[1])))
    x = T.dmatrix()
    xd = T.dot(x, d)
    evals = []
    evecs = []
    i,j = T.dscalars('i','j')
    for i in xrange(16):
        cost = T.dot(xd.T, xd) - np.sum(evals[j]*T.dot(evecs[j], d)*T.dot(evecs[j], d) for j in xrange(i))
        gd = T.grad(cost, d)
        y = d + 0.05*gd
        update_d = y / y.norm(2)
        train = theano.function(inputs=[x],outputs = [d], updates=[(d, update_d)])
        t = 0
        #tol = 0.005
        while t < 10: #and change in d < tol:
            train(X)
            t = t+1
        v = d.get_value()
        evecs.append(v)
        Xv = np.dot(X,v)
        la = np.dot(Xv.T,Xv)
        evals.append(la)
        
    evals = np.array(evals)
    evecs = np.array(evecs)

    D = evecs.T
    c= np.dot(evecs,X.T)
    
    #TODO: Write a code snippet that performs as indicated in the above comment
        
    for i in range(0, 200, 10):
        plot_reconstructions(D=D, c=c, num_coeff_array=[1, 2, 4, 6, 8, 10, 12, 14, 16], X_mean=X_mn.reshape((256, 256)), im_num=i)

    plot_top_16(D, 256, 'outputb/hw1b_top16_256.png')


if __name__ == '__main__':
    main()

In [56]:
evals

array([  3.91450369e+09,   1.51006420e+09,   8.75821262e+08,
         5.89224998e+08,   4.61892347e+08,   3.65738329e+08,
         3.29686630e+08,   2.74568334e+08,   2.37292433e+08,
         2.22708103e+08,   1.92926961e+08,   1.72007885e+08,
         1.63052120e+08,   1.49502569e+08,   1.19181209e+08,
         1.08308119e+08])

In [55]:
D.T.shape

(16, 65536)